
# Predict Student's Dropout And Academic Success
### Name: Sai Krishna Vamsi
### Reg No: 12001567


Dataset: <a href="https://www.kaggle.com/datasets/thedevastator/higher-education-predictors-of-student-retention">https://www.kaggle.com/datasets/thedevastator/higher-education-predictors-of-student-retention</a>

In [3]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from pylab import rcParams
import urllib
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import tree, model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,MaxAbsScaler
from sklearn import utils
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV
import scipy.stats as st

In [4]:
students = pd.read_csv('dataset.csv')

#In case I need to drop some columns that are skewing the results I create another df
students_reduced = students#.drop(columns = "Marital status")
#col is the number of columns in active df
col = 34
students.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [5]:
#We need 1 or 0 in the target column so I create an array and change dropout to 1 and graduate to 0 and replace it.
target=students_reduced.iloc[:,col]
target=np.array(target)                
for i in range(len(target)):
    if target[i] == "Dropout": target[i] = '1'
    if target[i] == "Graduate": target[i] = '0' 
students_reduced["Target"] = target

In [6]:
#In the target column there are also students that are enrolled and we dont need those in analysis so we drop them, as well
#as random numbers that got created along the way
students_reduced = students_reduced[students_reduced.Target != 'Enrolled'] 
students_reduced = students_reduced[(students_reduced.Target != '1')  | (students_reduced.Target != '0')]

In [7]:
#Next we set X as all the columns beside the target and y as the target
X = students_reduced.iloc[:, :-1].values
y = students_reduced.iloc[:, col].values

In [8]:
#We use the StandardScaler on the X data
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


In [9]:
#We use the train_test_split function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
#We use KNN function as the classifier
classifier = KNeighborsClassifier(n_neighbors=8, weights='distance')
#Then we fit X_train and y_train
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=8, weights='distance')

In [10]:
#To find the most effective parameter combination of KNeighborsClassifier we use grid_search where we set the Map first and 
#then we fit it on X and y axes
grid_search_params = {
    'n_neighbors' : [1,2,3,4,8,9,14,19,20,25],
    'weights' : ['uniform','distance'],
    'algorithm' : ['auto','ball_tree','kd_tree','brute']
    }
gs = GridSearchCV(classifier, grid_search_params, cv = 10, scoring="accuracy")
gs.fit(X,y)

GridSearchCV(cv=10,
             estimator=KNeighborsClassifier(n_neighbors=8, weights='distance'),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 2, 3, 4, 8, 9, 14, 19, 20, 25],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [11]:
#Last but not least we print out the best parameters and the best score which in this case is :
#Best parameters: KNeighborsClassifier(n_neighbors=8, weights='distance')
#Score : 0.8608815426997245
print("Best parameters:",gs.best_estimator_)
print("Best score :",gs.best_score_)

Best parameters: KNeighborsClassifier(n_neighbors=8, weights='distance')
Best score : 0.8608815426997245
